In [1]:
from bs4 import BeautifulSoup
import urllib
import re
import pandas as pd
import requests
from urllib.request import urlopen

In [2]:
def geturl(url):

    response = requests.get(url)
    page= response.text
    soup = BeautifulSoup(page, 'html.parser')    
    all_matches = soup.find_all('div', class_ = 'reviews')
    
    reviewer_type = soup.find_all('span', attrs = {'itemprop': 'name'})    
    reviewer = soup.find_all('span', attrs = {'itemprop': 'author'})   
    date = soup.find_all('span', attrs = {'itemprop': 'datePublished'})    
    review = soup.find_all('span', attrs = {'itemprop': 'description'})    
    star = soup.find_all('span', attrs = {'itemprop': 'ratingValue'})
    star = star[1:]
    
    dataframe = pd.DataFrame(
          {'reviewer_type': reviewer_type,
           'reviewer_name': reviewer,
           'date': date,
           'star': star,
           'review':review   
          })
    
    location = soup.find('div', attrs={'class': 'locality'}).text
    dataframe['location'] = location
    
    name_of_senior_living = soup.find('div', attrs={'class': 'name'}).text
    dataframe['name_of_senior_living'] = name_of_senior_living
    
    caring_type = soup.find('p', attrs = {'class': 'offers'}).text
    dataframe['type'] = caring_type
    
    return dataframe

In [3]:
def clean_bracket(dataset):
    
    dataset['reviewer_type'] = dataset['reviewer_type'].apply(str)
    dataset['reviewer_type'] = dataset['reviewer_type'].apply(lambda x: re.sub(r'\<span itemprop="name"\>', "", x))
    dataset['reviewer_type'] = dataset['reviewer_type'].apply(lambda x: re.sub(r'\<\/span\>', "", x))
    dataset['reviewer_name'] = dataset['reviewer_name'].apply(str)
    dataset['reviewer_name'] = dataset['reviewer_name'].apply(lambda x: re.sub(r'\<span itemprop="author"\>', "", x))
    dataset['reviewer_name'] = dataset['reviewer_name'].apply(lambda x: re.sub(r'\<\/span\>', "", x))
    dataset['date'] = dataset['date'].apply(str)
    dataset['date'] = dataset['date'].apply(lambda x: re.sub(r'\<span itemprop="datePublished"\>', "", x))
    dataset['date'] = dataset['date'].apply(lambda x: re.sub(r'\<\/span\>', "", x))
    dataset['star'] = dataset['star'].apply(str)
    dataset['star'] = dataset['star'].apply(lambda x: re.sub(r'\<span itemprop="ratingValue"\>', "", x))
    dataset['star'] = dataset['star'].apply(lambda x: re.sub(r'\<\/span\>', "", x))
    dataset['review'] = dataset['review'].apply(str)
    dataset['review'] = dataset['review'].apply(lambda x: re.sub(r'\<span itemprop="description"\>', "", x))
    dataset['review'] = dataset['review'].apply(lambda x: re.sub(r'\<\/span\>', "", x))
    
    return dataset

In [87]:
url_list=[
    'https://www.caring.com/senior-living/california/amador-county',
          'https://www.caring.com/senior-living/california/alameda-county',
          'https://www.caring.com/senior-living/california/butte-county',
          'https://www.caring.com/senior-living/california/calaveras-county',
            'https://www.caring.com/senior-living/california/colusa-county',
            'https://www.caring.com/senior-living/california/contra-costa-county',
            'https://www.caring.com/senior-living/california/el-dorado-county',
            'https://www.caring.com/senior-living/california/fresno-county',
            'https://www.caring.com/senior-living/california/glenn-county',
            'https://www.caring.com/senior-living/california/humboldt-county',
            'https://www.caring.com/senior-living/california/imperial-county',
            'https://www.caring.com/senior-living/california/kern-county',
            'https://www.caring.com/senior-living/california/kings-county',
            'https://www.caring.com/senior-living/california/lake-county',
            'https://www.caring.com/senior-living/california/los-angeles-county'
            'https://www.caring.com/senior-living/california/madera-county',
            'https://www.caring.com/senior-living/california/marin-county',
            'https://www.caring.com/senior-living/california/mendocino-county',
            'https://www.caring.com/senior-living/california/merced-county', # blank
            'https://www.caring.com/senior-living/california/monterey-county', 
            'https://www.caring.com/senior-living/california/napa-county', # blank
            'https://www.caring.com/senior-living/california/nevada-county',
            'https://www.caring.com/senior-living/california/orange-county',
            'https://www.caring.com/senior-living/california/placer-county',
            'https://www.caring.com/senior-living/california/riverside-county',
            'https://www.caring.com/senior-living/california/sacramento-county',
            'https://www.caring.com/senior-living/california/san-benito-county',
            'https://www.caring.com/senior-living/california/san-diego-county',
            'https://www.caring.com/senior-living/california/san-francisco-county',
            'https://www.caring.com/senior-living/california/san-joaquin-county',
            'https://www.caring.com/senior-living/california/san-luis-obispo-county',
            'https://www.caring.com/senior-living/california/san-mateo-county', # 404 not found
            'https://www.caring.com/senior-living/california/santa-barbara-county',
            'https://www.caring.com/senior-living/california/santa-clara-county',
            'https://www.caring.com/senior-living/california/santa-cruz-county',
            'https://www.caring.com/senior-living/california/shasta-county',
            'https://www.caring.com/senior-living/california/siskiyou-county',
            'https://www.caring.com/senior-living/california/solano-county',
            'https://www.caring.com/senior-living/california/sonoma-county',
            'https://www.caring.com/senior-living/california/stanislaus-county',
            'https://www.caring.com/senior-living/california/sutter-county',
            'https://www.caring.com/senior-living/california/tehama-county',
            'https://www.caring.com/senior-living/california/tulare-county',
            'https://www.caring.com/senior-living/california/tuolumne-county',
            'https://www.caring.com/senior-living/california/ventura-county', # blank
            'https://www.caring.com/senior-living/california/yolo-county',
            'https://www.caring.com/senior-living/california/yuba-county'
]

In [1]:
data = []
for url in url_list:
    response = requests.get(url)

    page = urlopen(url)
    soup = BeautifulSoup(page, 'lxml')
    all_matches = soup.findAll(attrs = {'class':['search-result']})

    

    for reviews in all_matches:
        review_url = reviews['data-link']

        df = geturl(review_url)
        data.append(df)

In [26]:
ca1 = pd.concat(data)
ca1

,reviewer_type,reviewer_name,date,star,review,location,name_of_senior_living,type
0,[I am a friend or relative of a current/past r...,[Lizzy49],[2018-07-07],[5],[This is a residence for mentally challenged m...,Plymouth,Schwabe Residential Home,Offers Assisted Living
0,[I visited this facility],[moomamma],[2014-03-06],[4],[I have heard such good comments about Jackson...,Jackson,Jackson Gardens Elder Care Home,Offers Memory Care
1,[I am/was a resident of this facility],[Biker_Momma45],[2014-03-05],[5],[I had my Dad in this facility recently and he...,Jackson,Jackson Gardens Elder Care Home,Offers Memory Care
2,[I am/was a resident of this facility],[solongdad],[2012-09-26],[3],[I found not all staff friendly. One nurse was...,Jackson,Jackson Gardens Elder Care Home,Offers Memory Care
0,[I am a friend or relative of a current/past r...,[Peggy],[2018-04-30],[5],"[Mom really likes Oak Manor, and everything ha...",Jackson,Oak Manor,Offers Assisted Living
...,...,...,...,...,...,...,...,...
3,[I am/was a resident of this facility],[Geneva301711],[2016-11-22],[5],"[My air conditioner went out, and Paul was on ...",Colusa,Eskaton Frank Jaconetti Manor,Offers Independent Living and Low Income Affor...
4,[I am/was a resident of this facility],[jgoodin],[2016-10-06],[5],[I’ve lived in other community housing on both...,Colusa,Eskaton Frank Jaconetti Manor,Offers Independent Living and Low Income Affor...
5,[I am/was a resident of this facility],[Georganna],[2016-10-06],[5],[I am so grateful to be part of Eskaton Frank ...,Colusa,Eskaton Frank Jaconetti Manor,Offers Independent Living and Low Income Affor...
0,[I visited this facility],[jennh378],[2017-05-22],[1],[The patients don't seem to be taken care of w...,Williams,Valley West Care Center,Offers Nursing Homes


In [29]:
ca2 = pd.concat(data)
ca2

,reviewer_type,reviewer_name,date,star,review,location,name_of_senior_living,type
0,[I visited this facility],[Jan],[2019-08-22],[3],[The apartment we saw at Carlton Senior Living...,Pleasant Hill,Carlton Senior Living Downtown Pleasant Hill,Offers Assisted Living and Memory Care
1,[I am a friend or relative of a current/past r...,[Don],[2017-11-27],[4],[My mother-in-law is in Carlton Senior Living ...,Pleasant Hill,Carlton Senior Living Downtown Pleasant Hill,Offers Assisted Living and Memory Care
2,[I visited this facility],[anna130536],[2017-05-31],[4],[We visited Carlton Senior Living. The rooms w...,Pleasant Hill,Carlton Senior Living Downtown Pleasant Hill,Offers Assisted Living and Memory Care
3,[I visited this facility],[william772237],[2017-05-31],[5],[Carlton Senior Living was very nice. I saw a ...,Pleasant Hill,Carlton Senior Living Downtown Pleasant Hill,Offers Assisted Living and Memory Care
4,[I visited this facility],[Katherine502873],[2017-04-12],[5],[I visited Carlton for my aunt. It was a wonde...,Pleasant Hill,Carlton Senior Living Downtown Pleasant Hill,Offers Assisted Living and Memory Care
...,...,...,...,...,...,...,...,...
0,[I am a friend or relative of a current/past r...,[Nell],[2017-11-20],[5],[Blades Roper Board &amp; Care takes very good...,Los Angeles,Blades Roper Board & Care,Offers Assisted Living
0,[I visited this facility],[Ysavelle],[2017-01-26],[5],[Its a best place to your loved ones and so qu...,Rancho Palos Verdes,Avenida Villa,Offers Assisted Living
0,[I am a friend or relative of a current/past r...,[Linda836575],[2017-01-14],[5],[She’s been in Jennifer’s Home for about a mon...,Newhall,Jennifer Home,Offers Assisted Living
0,[I visited this facility],[Anne],[2020-03-25],[5],[West Hills Home Care II was very nice. I felt...,Los Angeles,West Hills Home Care II,Offers Assisted Living


In [39]:
ca3 = pd.concat(data)
ca3

,reviewer_type,reviewer_name,date,star,review,location,name_of_senior_living,type
0,[I am a friend or relative of a current/past r...,[F.O.],[2018-08-20],[4],"[My wife has been at Cedar Creek, but since th...",Madera,Cedar Creek Senior Living,"Offers Assisted Living, Independent Living, an..."
1,[I visited this facility],[Donna],[2017-10-16],[5],[Cedar Creek was beautiful. I would actually l...,Madera,Cedar Creek Senior Living,"Offers Assisted Living, Independent Living, an..."
2,[I visited this facility],[John167485],[2017-03-30],[5],[I was impressed with Cedar Creek. It looked v...,Madera,Cedar Creek Senior Living,"Offers Assisted Living, Independent Living, an..."
3,[I am a friend or relative of a current/past r...,[HisGrandChild1],[2017-01-16],[2],[They are good for people who are independent....,Madera,Cedar Creek Senior Living,"Offers Assisted Living, Independent Living, an..."
4,[I am a friend or relative of a current/past r...,[Fred676668],[2016-12-13],[5],[I visited Cedar Creek Senior Living with my s...,Madera,Cedar Creek Senior Living,"Offers Assisted Living, Independent Living, an..."
...,...,...,...,...,...,...,...,...
1,[I am a friend or relative of a resident],[Caring105949250],[2015-05-08],[4],[My mother went twice to Novato Healthcare a m...,Novato,Country Villa Novato Healthcare Center,Offers Nursing Homes
2,[I am a friend or relative of resident],[RICHCALZ],[2014-07-17],[5],[my wife was in the dementia unit for 3 1/2 ye...,Novato,Country Villa Novato Healthcare Center,Offers Nursing Homes
3,[I am a friend or relative of resident],[furious daughter],[2014-02-14],[2],[My father fell and fractured his spine in May...,Novato,Country Villa Novato Healthcare Center,Offers Nursing Homes
0,[Other],[DrJames'Corner],[2011-09-01],[3],[Kindred Healthcare - Hospice Care Center prov...,San Rafael,San Rafael Healthcare and Wellness,Offers Nursing Homes


In [44]:
ca4 = pd.concat(data)
ca4

,reviewer_type,reviewer_name,date,star,review,location,name_of_senior_living,type
0,[I am a friend or relative of resident],[Tammy Bray],[2015-05-04],[5],[We are very pleased with the care that Sherwo...,Fort Bragg,Sherwood Oaks Health Center,Offers Nursing Homes
0,[I am a friend or relative of a current/past r...,[Charlene Light],[2017-09-29],[5],[My parents lived at Brookside from 2008 to 20...,Ukiah,Brookside Retirement Residence,Offers Independent Living and Continuing Care ...
1,[I am/was a resident of this facility],[Elizabeth],[2017-07-17],[5],[The people at Brookside Retirement Residence ...,Ukiah,Brookside Retirement Residence,Offers Independent Living and Continuing Care ...
2,[I am a friend or relative of a resident],[Caring84735550],[2014-07-09],[4],[My mom just moved into Brookside Retirement R...,Ukiah,Brookside Retirement Residence,Offers Independent Living and Continuing Care ...
0,[I visited this facility],[Maria],[2018-02-28],[5],"[Mountain View was pretty expensive, but we re...",Ukiah,Mountain View,Offers Assisted Living
1,[I am a friend or relative of a resident],[Diane109424750],[2015-07-06],[4],[Mountain View is fine. The apartments are ver...,Ukiah,Mountain View,Offers Assisted Living
2,[I visited this facility],[Glen3],[2014-07-31],[4],[I toured Mountain View. The grounds were well...,Ukiah,Mountain View,Offers Assisted Living
3,[I am a friend or relative of a resident],[Dena1],[2013-08-26],[4],[I chose this facility because I live in Ukiah...,Ukiah,Mountain View,Offers Assisted Living
4,[I am a friend or relative of a resident],[scoobydo17],[2012-09-26],[4],[My grandmother is staying at Mountain View Se...,Ukiah,Mountain View,Offers Assisted Living
5,[I am a friend or relative of a resident],[clubcake],[2012-07-05],[4],[When I visited my grandmother there on severa...,Ukiah,Mountain View,Offers Assisted Living


In [56]:
ca5 = pd.concat(data)
ca5

,reviewer_type,reviewer_name,date,star,review,location,name_of_senior_living,type
0,[I am a friend or relative of a current/past r...,[JTarman],[2020-04-19],[5],[Great staff and accommodations. I visit frequ...,Monterey,The Park Lane,"Offers Assisted Living, Independent Living, an..."
1,[I visited this facility],[Daugter of a Mom at Park Lane],[2020-04-18],[5],[We have been pleased with both the staff and ...,Monterey,The Park Lane,"Offers Assisted Living, Independent Living, an..."
2,[I am a friend or relative of a current/past r...,[Rufbud],[2020-04-18],[5],[The assistance that we were given when we fir...,Monterey,The Park Lane,"Offers Assisted Living, Independent Living, an..."
3,[I am a friend or relative of a current/past r...,[patpatf],[2020-04-18],[5],[Everyone is very helpful and feels like famil...,Monterey,The Park Lane,"Offers Assisted Living, Independent Living, an..."
4,[I am a friend or relative of a current/past r...,[JimmyinPG],[2020-04-18],[5],[We've been very impressed with TPL. My mom h...,Monterey,The Park Lane,"Offers Assisted Living, Independent Living, an..."
...,...,...,...,...,...,...,...,...
1,[I visited this facility],[Jan],[2018-01-12],[2],[Queen Of Angels II RCFE is a house. The floor...,Salinas,Queen Of Angels Ii Rcfe,Offers Assisted Living
0,[I am/was a resident of this facility],[isiah martinez],[2018-01-26],[5],"[This was a great place, i was treated with re...",Monterey,Windsor Monterey Care Center,Offers Nursing Homes
1,[I am a friend or relative of resident],[lncv333],[2015-02-19],[1],[My relative has been at Windsor Monterey for ...,Monterey,Windsor Monterey Care Center,Offers Nursing Homes
0,[I am a friend or relative of a current/past r...,[Kathy],[2018-04-03],[3],[My mom stayed at Katherine Healthcare Center....,Salinas,Katherine Healthcare Center,Offers Nursing Homes


In [62]:
ca6 = pd.concat(data)
ca6

,reviewer_type,reviewer_name,date,star,review,location,name_of_senior_living,type
0,[I visited this facility],[Joanna],[2019-05-28],[5],[Eskaton Village Grass Valley assisted living ...,Grass Valley,Eskaton Village Grass Valley,"Offers Assisted Living, Independent Living, an..."
1,[I am a friend or relative of a current/past r...,[Michael],[2018-08-10],[4],[Eskaton Village Grass Valley was clean and pr...,Grass Valley,Eskaton Village Grass Valley,"Offers Assisted Living, Independent Living, an..."
2,[Other],[Emily T],[2018-02-22],[5],"[My five stars are for the sales office, the c...",Grass Valley,Eskaton Village Grass Valley,"Offers Assisted Living, Independent Living, an..."
3,[I am/was a resident of this facility],[Lilian],[2017-09-26],[5],[I love it here. I've been here for 8 years. T...,Grass Valley,Eskaton Village Grass Valley,"Offers Assisted Living, Independent Living, an..."
4,[I am/was a resident of this facility],[Eunice],[2017-09-24],[5],[They treat us well. They are very courteous. ...,Grass Valley,Eskaton Village Grass Valley,"Offers Assisted Living, Independent Living, an..."
...,...,...,...,...,...,...,...,...
3,[Visited this facility],[looking out for others in diffucult times],[2011-04-27],[1],[Horrible! Never put a loved one in this home...,Roseville,Pine Creek Care Center,Offers Nursing Homes
0,[I am a friend or relative of a resident],[Caring1185],[2013-05-30],[3],[My daughter-in-law had brain surgery and she ...,Roseville,Sutter Transitional and Respiratory Services,Offers Nursing Homes
0,[I visited this facility],[Janet22],[2013-07-10],[3],"[Woodcreek Terrace was okay, it's backed up ag...",Roseville,Woodcreek Terrace,Offers Independent Living
0,[I visited this facility],[Karen],[2017-08-24],[3],"[I went to Creekside, it's a very small facili...",Auburn,Auburn Creekside Villa,Offers Assisted Living


In [65]:
ca7 = pd.concat(data)
ca7

,reviewer_type,reviewer_name,date,star,review,location,name_of_senior_living,type
0,[I am/was a resident of this facility],[Macgyver17],[2020-03-08],[5],[Very friendly staff excellent services. They...,Hemet,The Camelot,Offers Independent Living
1,[I visited this facility],[D.A.],[2019-12-15],[3],[The Camelot looks like a big castle. I didn't...,Hemet,The Camelot,Offers Independent Living
2,[I am/was a resident of this facility],[Michael],[2019-11-01],[5],[I was at The Camelot for two years. It was gr...,Hemet,The Camelot,Offers Independent Living
3,[I visited this facility],[Judee],[2019-05-10],[4],"[Camelot was OK, nice, and clean. The room was...",Hemet,The Camelot,Offers Independent Living
4,[I visited this facility],[CH],[2018-09-06],[4],[The Camelot was a great place and it's very n...,Hemet,The Camelot,Offers Independent Living
...,...,...,...,...,...,...,...,...
2,[I am/was a resident of this facility],[HP586834],[2016-11-09],[1],[I am not really satisfied with Overture. The ...,Riverside,Overture Riverwalk 55+ Apartment Homes,Offers Independent Living
0,[I am a friend or relative of a current/past r...,[scubeedoi],[2018-12-05],[1],[Run.....Not what it once was \r\nAlmost all t...,La Quinta,Rachel's Cottage with Love,Offers Assisted Living
1,[I visited this facility],[Carmen720227],[2016-10-05],[5],[I was very happy with Rachel’s Cottage. It’s ...,La Quinta,Rachel's Cottage with Love,Offers Assisted Living
0,[I visited this facility],[Carol],[2020-03-24],[3],[I didn't care for At Sunny Hills Home Care. T...,Moreno Valley,At Sunny Hills Home Care,Offers Assisted Living


In [68]:
ca8 = pd.concat(data)
ca8

,reviewer_type,reviewer_name,date,star,review,location,name_of_senior_living,type
0,[I am a friend or relative of a current/past r...,[HM],[2019-05-07],[5],[Atria El Camino Gardens was a better facility...,Carmichael,Atria El Camino Gardens,"Offers Assisted Living, Independent Living, an..."
1,[I am a friend or relative of a current/past r...,[Fred],[2019-04-19],[4],[My dad is doing respite at Atria El Camino Ga...,Carmichael,Atria El Camino Gardens,"Offers Assisted Living, Independent Living, an..."
2,[I am a friend or relative of a current/past r...,[Cindy],[2018-10-18],[5],[We're very happy with Atria El Camino Gardens...,Carmichael,Atria El Camino Gardens,"Offers Assisted Living, Independent Living, an..."
3,[I visited this facility],[Rick],[2018-03-14],[5],"[Atria El Camino Gardens was nice, very clean,...",Carmichael,Atria El Camino Gardens,"Offers Assisted Living, Independent Living, an..."
4,[I visited this facility],[marci342365],[2017-05-31],[4],[Atria El Camino Gardens was a great place; th...,Carmichael,Atria El Camino Gardens,"Offers Assisted Living, Independent Living, an..."
...,...,...,...,...,...,...,...,...
1,[I visited this facility],[Gina],[2018-07-26],[5],[Brookside Skilled Nursing Hospital was very w...,San Mateo,Brookside Skilled Nursing Hospital,Offers Nursing Homes
2,[I am a friend or relative of a current/past r...,[at411410],[2017-05-31],[5],[Brookside Skilled Nursing Hospital was a grea...,San Mateo,Brookside Skilled Nursing Hospital,Offers Nursing Homes
3,[I am a friend or relative of a resident],[Caroline11],[2015-01-12],[3],[My mother-in-law is in a nursing home. Everyb...,San Mateo,Brookside Skilled Nursing Hospital,Offers Nursing Homes
4,[I am a friend or relative of resident],[fratello],[2015-01-05],[2],[this facility my be ok for persons that can t...,San Mateo,Brookside Skilled Nursing Hospital,Offers Nursing Homes


In [71]:
ca9 = pd.concat(data)
ca9

,reviewer_type,reviewer_name,date,star,review,location,name_of_senior_living,type
0,[I am/was a resident of this facility],[Shirlee],[2019-10-09],[4],[I moved into Santa Maria Terrace and they hav...,Santa Maria,Santa Maria Terrace,Offers Assisted Living and Independent Living
1,[I am a friend or relative of a current/past r...,[Julia],[2019-01-16],[4],[My ex-husband has moved into Santa Maria Terr...,Santa Maria,Santa Maria Terrace,Offers Assisted Living and Independent Living
2,[I visited this facility],[Deborah731151],[2017-03-27],[5],"[Santa Maria Terrace used to be a hotel, so it...",Santa Maria,Santa Maria Terrace,Offers Assisted Living and Independent Living
3,[I visited this facility],[John P.484128],[2017-02-07],[4],"[Santa Maria was nice, not that fancy, and loo...",Santa Maria,Santa Maria Terrace,Offers Assisted Living and Independent Living
4,[I visited this facility],[Elise526036],[2016-12-05],[4],"[Santa Maria Terrace seemed a very nice place,...",Santa Maria,Santa Maria Terrace,Offers Assisted Living and Independent Living
...,...,...,...,...,...,...,...,...
1,[I visited this facility],[James],[2019-04-14],[4],[The Gardens' memory care was quite depressing...,Yuba City,The Gardens Alzheimer's & Special Care,Offers Adult Day Care and Memory Care
0,[I visited this facility],[unity counts],[2015-02-05],[4],[The line and janitorial staff work really har...,Yuba City,Willow Glen Care Center,Offers Assisted Living
1,[I am a friend or relative of a resident],[Caring0249],[2014-10-25],[4],"[Willow Glen was excellent, the staff was grea...",Yuba City,Willow Glen Care Center,Offers Assisted Living
2,[I am a friend or relative of resident],[Judith Dale],[2014-03-03],[4],[My sister has been at Willow Glen for about a...,Yuba City,Willow Glen Care Center,Offers Assisted Living


In [77]:
ca10 = pd.concat(data)
ca10

,reviewer_type,reviewer_name,date,star,review,location,name_of_senior_living,type
0,[I visited this facility],[Debbi],[2019-04-19],[5],[Brookdale Red Bluff was beautiful. They took ...,Red Bluff,Brookdale Red Bluff,Offers Assisted Living and Memory Care
1,[I visited this facility],[Cheryal],[2017-08-21],[5],[Brookdale Red Bluff was very nice and it's so...,Red Bluff,Brookdale Red Bluff,Offers Assisted Living and Memory Care
2,[I visited this facility],[ChrisGPHJV],[2016-05-10],[4],[We went to Brookdale Red Bluff. My mom lived ...,Red Bluff,Brookdale Red Bluff,Offers Assisted Living and Memory Care
3,[I am a friend or relative of a resident],[GeneralChicken],[2012-06-29],[4],[It seemed to me that for the situation the fa...,Red Bluff,Brookdale Red Bluff,Offers Assisted Living and Memory Care
0,[I visited this facility],[Michael],[2020-04-13],[4],[Red Bluff Senior Living is older than ther ot...,Red Bluff,Red Bluff Senior Living,Offers Independent Living
...,...,...,...,...,...,...,...,...
3,[I visited this facility],[Terry],[2017-08-21],[4],[Prestige Assisted Living at Marysville was be...,Marysville,Prestige Assisted Living at Marysville,Offers Assisted Living and Memory Care
4,[I am a friend or relative of a current/past r...,[Sally055357],[2016-09-30],[4],"[Prestige was really nice, and I liked it, but...",Marysville,Prestige Assisted Living at Marysville,Offers Assisted Living and Memory Care
5,[I am a friend or relative of a resident],[Deni1],[2014-06-27],[5],[Very caring management and staff. Professiona...,Marysville,Prestige Assisted Living at Marysville,Offers Assisted Living and Memory Care
6,[Other],[sandra9],[2012-09-17],[5],[hhaving been an ombudsman for the county of S...,Marysville,Prestige Assisted Living at Marysville,Offers Assisted Living and Memory Care


In [83]:
ca11 = pd.concat(data)
ca11

,reviewer_type,reviewer_name,date,star,review,location,name_of_senior_living,type
0,[I am a friend or relative of a current/past r...,[Karen],[2019-03-29],[4],[My parents are in The Hampshire. The building...,Merced,The Hampshire,Offers Independent Living
1,[I visited this facility],[Dink],[2018-08-10],[5],[I visited The Hampshire and Sandy. The lady t...,Merced,The Hampshire,Offers Independent Living
2,[I am a friend or relative of a current/past r...,[Retha],[2017-12-29],[5],[My son moved into independent living of The H...,Merced,The Hampshire,Offers Independent Living
3,[I visited this facility],[Christina],[2017-07-17],[5],[Everything at The Hampshire was excellent. I ...,Merced,The Hampshire,Offers Independent Living
4,[I am/was a resident of this facility],[Margaret728000],[2016-11-02],[5],[I like The Hampshire very well. I have moved ...,Merced,The Hampshire,Offers Independent Living
5,[I visited this facility],[Gary816675],[2016-10-06],[5],"[Of all the facilities that I visited, I spent...",Merced,The Hampshire,Offers Independent Living
6,[I visited this facility],[Aida112055],[2016-06-10],[5],[The Hampshire was so nice. The guy in charge ...,Merced,The Hampshire,Offers Independent Living
7,[I visited this facility],[Elizabeth115030350],[2015-11-15],[5],[The Hampshire was very nice and cheerful. Eve...,Merced,The Hampshire,Offers Independent Living
8,[I visited this facility],[Sally34],[2015-06-28],[4],"[The Hampshire was well run, and it smelled go...",Merced,The Hampshire,Offers Independent Living
9,[I visited this facility],[Caring92159250],[2014-08-29],[5],[I had an appointment in Hampshire at 1 p.m. a...,Merced,The Hampshire,Offers Independent Living


In [86]:
ca12 = pd.concat(data)
ca12

,reviewer_type,reviewer_name,date,star,review,location,name_of_senior_living,type
0,[I am a friend or relative of a current/past r...,[M.N.],[2020-01-15],[5],[My brother moved to Redwood Retirement Reside...,Napa,Redwood Retirement Residence,Offers Independent Living
1,[I visited this facility],[Susan],[2019-08-24],[5],"[Redwood Retirement Residence was lovely, but ...",Napa,Redwood Retirement Residence,Offers Independent Living
2,[I visited this facility],[Gagen],[2019-07-15],[4],[Redwood Retirement Residence was nice. I like...,Napa,Redwood Retirement Residence,Offers Independent Living
3,[I visited this facility],[Harold],[2019-01-25],[3],[Redwood Retirement Residence was attractive o...,Napa,Redwood Retirement Residence,Offers Independent Living
4,[I visited this facility],[John],[2018-07-30],[5],[Redwood Retirement Residence is a very nice c...,Napa,Redwood Retirement Residence,Offers Independent Living
5,[I visited this facility],[Reviewer],[2018-02-27],[3],[Redwood Retirement was nice. The staff was ve...,Napa,Redwood Retirement Residence,Offers Independent Living
6,[I visited this facility],[Susan],[2017-07-17],[4],[The rooms at Redwood Retirement Residence wer...,Napa,Redwood Retirement Residence,Offers Independent Living
7,[I visited this facility],[Dennis205874],[2017-01-06],[3],"[I thought Redwood was a good, nice, and clean...",Napa,Redwood Retirement Residence,Offers Independent Living
8,[I am/was a resident of this facility],[Mary167410],[2016-12-30],[4],[I moved into Redwood Retirement with some of ...,Napa,Redwood Retirement Residence,Offers Independent Living
9,[I visited this facility],[Tom207278],[2016-11-29],[5],[Redwood was very good. We had a good time the...,Napa,Redwood Retirement Residence,Offers Independent Living


In [95]:
ca13 = pd.concat(data)
ca13

,reviewer_type,reviewer_name,date,star,review,location,name_of_senior_living,type
0,[I am a friend or relative of a current/past r...,[Ron],[2019-07-08],[5],[The staff at Brookdale Camarillo treated my f...,Camarillo,Brookdale Camarillo,"Offers Assisted Living, Nursing Homes, and Mem..."
1,[I visited this facility],[Priscilla],[2019-06-15],[5],[The rooms at Brookdale Camarillo were really ...,Camarillo,Brookdale Camarillo,"Offers Assisted Living, Nursing Homes, and Mem..."
2,[I visited this facility],[Jeff],[2019-01-13],[5],[Brookdale Camarillo was very nice and really ...,Camarillo,Brookdale Camarillo,"Offers Assisted Living, Nursing Homes, and Mem..."
3,[I visited this facility],[Susan],[2018-05-08],[3],[Brookdale Camarillo is a very good facility. ...,Camarillo,Brookdale Camarillo,"Offers Assisted Living, Nursing Homes, and Mem..."
4,[I visited this facility],[Gail],[2017-08-21],[4],[Brookdale Camarillo was nice. I really liked ...,Camarillo,Brookdale Camarillo,"Offers Assisted Living, Nursing Homes, and Mem..."
...,...,...,...,...,...,...,...,...
4,[I am a friend or relative of a resident],[Marie43],[2015-03-31],[4],[My ex-husband is in Carmen's Care Manor. It i...,Simi Valley,Carmen's Care Manor #1,Offers Assisted Living
0,[I am a friend or relative of a current/past r...,[Visitor847410],[2017-02-10],[4],[Charter Oak was very good. I looked at it and...,Simi Valley,Family First Home Care,Offers Assisted Living
0,[I visited this facility],[Priscilla],[2019-06-15],[2],[Ventura Grand Chateau was very small. The fac...,Ventura,Ventura Grand Chateau,Offers Assisted Living and Memory Care
1,[I am a friend or relative of a current/past r...,[Angie],[2017-12-27],[4],[My husband is at the Mound Guest Home. They h...,Ventura,Ventura Grand Chateau,Offers Assisted Living and Memory Care


In [90]:
ca = ca1.append([ca2, ca3, ca4, ca5, ca6, ca7, ca8, ca9, ca10, ca11, ca12, ca13])
ca = clean_bracket(ca)
ca


,reviewer_type,reviewer_name,date,star,review,location,name_of_senior_living,type
0,I am a friend or relative of a current/past re...,Lizzy49,2018-07-07,5,This is a residence for mentally challenged me...,Plymouth,Schwabe Residential Home,Offers Assisted Living
0,I visited this facility,moomamma,2014-03-06,4,I have heard such good comments about Jackson ...,Jackson,Jackson Gardens Elder Care Home,Offers Memory Care
1,I am/was a resident of this facility,Biker_Momma45,2014-03-05,5,I had my Dad in this facility recently and he ...,Jackson,Jackson Gardens Elder Care Home,Offers Memory Care
2,I am/was a resident of this facility,solongdad,2012-09-26,3,I found not all staff friendly. One nurse was ...,Jackson,Jackson Gardens Elder Care Home,Offers Memory Care
0,I am a friend or relative of a current/past re...,Peggy,2018-04-30,5,"Mom really likes Oak Manor, and everything has...",Jackson,Oak Manor,Offers Assisted Living
...,...,...,...,...,...,...,...,...
4,I am a friend or relative of a resident,Marie43,2015-03-31,4,My ex-husband is in Carmen's Care Manor. It is...,Simi Valley,Carmen's Care Manor #1,Offers Assisted Living
0,I am a friend or relative of a current/past re...,Visitor847410,2017-02-10,4,Charter Oak was very good. I looked at it and ...,Simi Valley,Family First Home Care,Offers Assisted Living
0,I visited this facility,Priscilla,2019-06-15,2,Ventura Grand Chateau was very small. The faci...,Ventura,Ventura Grand Chateau,Offers Assisted Living and Memory Care
1,I am a friend or relative of a current/past re...,Angie,2017-12-27,4,My husband is at the Mound Guest Home. They he...,Ventura,Ventura Grand Chateau,Offers Assisted Living and Memory Care


In [91]:
ca.to_csv('caring_data.csv')
